In [2]:
import faiss
import numpy as np
import pandas as pd

# Load embeddings from your CSV or PKL
pkl_path = "C:\\HAP774\\New folder (2)\\MedLlama_Chatbot\\cleaned_data_with_embeddings.pkl"  # Replace with your file path
df = pd.read_pickle(pkl_path)

# Extract embeddings into a numpy array
embeddings = np.vstack(df["embeddings"].values)

In [3]:
# Define a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # Use L2 distance
index.add(embeddings)  # Add embeddings to the index
print(f"Number of embeddings in the index: {index.ntotal}")


Number of embeddings in the index: 256916


In [4]:
# Save the FAISS index for later use
faiss.write_index(index, "faiss_index.idx")

In [5]:
from sentence_transformers import SentenceTransformer

# Load FAISS index and the embedding model
index = faiss.read_index("faiss_index.idx")
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\HAP774\New folder (2)\MedLlama_Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Function to handle queries
def chatbot_query(user_query, df, top_k=5):
    # Convert query to embedding
    query_embedding = model.encode(user_query).reshape(1, -1)
    
    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve top results
    results = []
    for i, idx in enumerate(indices[0]):
        result = {
            "rank": i + 1,
            "description": df.iloc[idx]["Description"],
            "patient": df.iloc[idx]["Patient"],
            "response": df.iloc[idx]["Doctor"],
            "distance": distances[0][i],
        }
        results.append(result)
    
    return results

In [ ]:
# Test the chatbot
query = "I have a headache, what can i do?"
results = chatbot_query(query, df)
for res in results:
    print(f"Rank: {res['rank']}, Distance: {res['distance']}")
    print(f"Question: {res['description']}")
    print(f"Patient Query: {res['patient']}")
    print(f"Response: {res['response']}\n")

Rank: 1, Distance: 0.69089674949646
Question: Q. What should I do for persistent headache?
Patient Query: Hi doctor, Initially, my problem started with a headache, and I felt shaky and dizzy the next day. I felt the same for around four days and went to the hospital. They said that I was dehydrated. But they did not give me anything for it and sent me home. I still have a headache, shaky, dizzy and feel almost worse. What should I do?
Response: Hi. I am sorry to hear about your persistent troubles. For further information consult an internal medicine physician online

Rank: 2, Distance: 0.700675368309021
Question: Suggest treatment for severe headache for past 2 months
Patient Query: I have been having a sharp pain in my head for the past month-2 months, and I thought it was maybe because I was dehydrated, but continuing to drink water and stay hydrated didn t help, neither did Tylenol, or Aleve, and Advil Doesn t work for me for anything, so I didn t bother trying that. So do you have